# Cosmological Power Spectra Calculation
This code calculates and stores cosmological power spectra 
and related coefficients for weak lensing and matter power spectra based on cosmological models.

In [1]:
import os
import json
import numpy
import pyccl
import scipy

In [ ]:
# FOLDER
FOLDER = '/global/cfs/cdirs/lsst/groups/MCP/CosmoCloud/LimberCloud/'
INFO_FOLDER = os.path.join(FOLDER, 'INFO')

Choosing a cosmological model.

In [ ]:
# Cosmology
with open(os.path.join(INFO_FOLDER, 'COSMOLOGY.json'), 'r') as file:
    COSMOLOGY_INFO = json.load(file)

COSMOLOGY = pyccl.Cosmology(
    h = COSMOLOGY_INFO['H'],
    w0 = COSMOLOGY_INFO['W0'],
    wa = COSMOLOGY_INFO['WA'], 
    n_s = COSMOLOGY_INFO['NS'], 
    A_s = COSMOLOGY_INFO['AS'],
    m_nu = COSMOLOGY_INFO['M_NU'],  
    Neff = COSMOLOGY_INFO['N_EFF'],
    Omega_b = COSMOLOGY_INFO['OMEGA_B'], 
    Omega_k = COSMOLOGY_INFO['OMEGA_K'], 
    Omega_c = COSMOLOGY_INFO['OMEGA_CDM'], 
    mass_split = 'single', matter_power_spectrum = 'halofit', transfer_function = 'boltzmann_camb',
    extra_parameters = {'camb': {'kmax': 50, 'lmax': 5000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

Calculate the power spectrum.

In [ ]:
# Define the redshift grid
Z1 = 0.0
Z2 = 3.5
GRID_SIZE = 350
Z_GRID = numpy.linspace(Z1, Z2, GRID_SIZE + 1)

# Calculate corresponding scale factors and comoving distances
A_GRID = 1 / (1 + Z_GRID)
CHI_GRID = pyccl.background.comoving_radial_distance(cosmo=COSMOLOGY, a=A_GRID)

CHI_SIZE = 500
Z_DATA = numpy.linspace(Z1, Z2, CHI_SIZE + 1)

A_DATA = 1 / (1 + Z_DATA)
CHI_DATA = pyccl.background.comoving_radial_distance(cosmo=COSMOLOGY, a=A_DATA)

ELL1 = 20
ELL2 = 2000
ELL_SIZE = 20
ELL_GRID = numpy.geomspace(ELL1, ELL2, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_GRID)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out=numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where=CHI_MESH > 0))

# Calculate the power spectrum on the grid
POWER_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):
    POWER_GRID[:,GRID_INDEX] = pyccl.power.nonlin_matter_power(cosmo=COSMOLOGY, k=SCALE_GRID[:,GRID_INDEX], a=A_GRID[GRID_INDEX])

Compute the numerical integral needed for the covariance matrix.

In [5]:
# Numerical integral

def integral_I2(chi1, chi2, power1, power2):
    
    def formula_0(chi):
        result = (chi2 - chi) / (chi * (chi2 - chi1))
        
        result = result * (chi - chi1) /(chi * (chi2 - chi1))
        
        result = result * (((chi / chi2) ** 3) * power2) 
        
        return result   
    
    def formula_n(chi):    
        result = (chi2 - chi) / (chi * (chi2 - chi1))
        
        result = result * (chi-chi1) / (chi * (chi2 - chi1))
        
        result = result * ((chi2 - chi) / (chi2 - chi1) * power1 + (chi - chi1) / (chi2 - chi1) * power2) 
        
        return result
    
    if chi1 == 0:
        integral = scipy.integrate.quad_vec(f=formula_0, a=chi1, b=chi2)[0]
    else:
        integral = scipy.integrate.quad_vec(f=formula_n, a=chi1, b=chi2)[0]
    return integral

Calculate the coefficients that are used in the angular power spectrum computation.

In [6]:
# Coefficient

def coefficient_J2(chi1, chi2, power1, power2):
    a = 1 - chi1 / chi2
    p = 1 - power1 / power2
    
    if a == 1:
        formula = 1 / 12
    else: 
        formula = (( - a) * ( - 6 * p + a * (4 + p)) + 2 * (a ** 2 + 3 * p - 2 * a * (1 + p)) * numpy.log(1 - a)) / (2 * a ** 3)
    
    coefficient = power2 * formula / chi2
    return coefficient


In [7]:
# Case1: chi1 > 0

ELL_INDEX = 0
GRID_INDEX = int(GRID_SIZE / 2)

CHI1 = numpy.float128(CHI_GRID[GRID_INDEX])
CHI2 = numpy.float128(CHI_GRID[GRID_INDEX + 1])

POWER1 = numpy.float128(POWER_GRID[ELL_INDEX, GRID_INDEX])
POWER2 = numpy.float128(POWER_GRID[ELL_INDEX, GRID_INDEX + 1])

REDSHIFT1 = numpy.float128(Z_GRID[GRID_INDEX])
REDSHIFT2 = numpy.float128(Z_GRID[GRID_INDEX + 1])

print(CHI1, CHI2, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

4924.221744025123371 4940.716645699534638 11370.147631619063759 11268.969970366384587 1.75 1.7600000000000000089


Compute the integral and coefficient for the case where chi1 > 0

In [8]:
# Case1: chi1 > 0

INTEGRAL = integral_I2(CHI1, CHI2, POWER1, POWER2)

COEFFICIENT = coefficient_J2(CHI1, CHI2, POWER1, POWER2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

0.0012790904134415704096 0.0012790904134418623329 2.2822726781362989712e-13


In [9]:
# Case2: chi1 = 0

ELL_INDEX = 0
GRID_INDEX = 0

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]

POWER1 = POWER_GRID[ELL_INDEX, GRID_INDEX]
POWER2 = POWER_GRID[ELL_INDEX, GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

0.0 44.40057919571794 0.0 1826.8838600584127 0.0 0.01


Compute the integral and coefficient for the case where chi1 = 0

In [10]:
# Case2: chi1 = 0

INTEGRAL = integral_I2(CHI1, CHI2, POWER1, POWER2)

COEFFICIENT = coefficient_J2(CHI1, CHI2, POWER1, POWER2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

3.4287913452763403 3.42879134527634 -1.1102230246251565e-16
